In [93]:
%matplotlib inline

# Author: Xiang Wang

from __future__ import print_function, division

import os
import copy

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms, models

import numpy as np
import matplotlib.pyplot as plt
import time


plt.ion()   # interactive mode

## 一、文件夹结构设计和数据存放

### 1. 建立存放数据的文件夹

In [49]:
# shutil －一种高层次的文件操作工具，强大之处是在对文件的复制与删除操作支持比较好
import os
import shutil

In [ ]:
# 建立data文件夹用于存放数据，train和val分别存放训练集和验证集
# .mkdir的函数用于建立该路径下的文件夹
os.mkdir('D:/1.DataAnalysisPrograme/DeepLearning/Pytroch/Data/CIFAR/')
os.mkdir('D:/1.DataAnalysisPrograme/DeepLearning/Pytroch/Data/CIFAR/train')
os.mkdir('D:/1.DataAnalysisPrograme/DeepLearning/Pytroch/Data/CIFAR/val')

In [ ]:
# 得到id和label，以便以下面分割数据集
import pandas as pd

idLabel = pd.read_csv('D:/1.DataAnalysisPrograme/DeepLearning/Pytroch/Data/CIFAR/trainLabels.csv')

In [ ]:
# 创建训练集分类的label的文件夹
train_root = 'D:/1.DataAnalysisPrograme/DeepLearning/Pytroch/Data/CIFAR/train'

for label in idLabel.label.unique():
    os.mkdir(os.path.join(train_root, label))

In [ ]:
# 创建测试集分类的label的文件夹
val_root = 'D:/1.DataAnalysisPrograme/DeepLearning/Pytroch/Data/CIFAR/val'

for label in idLabel.label.unique():
    os.mkdir(os.path.join(val_root, label))

### 2. 分割数据集并将其存入对应文件夹


In [ ]:
# .listdir主要用于列出该路径下所有的文件的名称，以列表的形式得到
# 这里的路径是下载下来的CIFAR-10的训练集数据存放的路径
data_file = os.listdir('D:/1.DataAnalysisPrograme/DeepLearning/Pytroch/Data/CIFAR-10/zip/train')

In [ ]:
# 分割数据集
import numpy as np
idLabel_train = np.array(idLabel[['id','label']])[:48000]
idLabel_val = np.array(idLabel[['id','label']])[48000:]

In [ ]:
# 移动图片在训练集的相应位置
root = 'D:/1.DataAnalysisPrograme/DeepLearning/Pytroch/Data/CIFAR-10/'

for (id,label) in idLabel_train:
    
    pic_path = root + 'zip/train/' + str(id) + '.png'
    
    obj_path = train_root + '/' + str(label) + '/' + str(id) + '.png'
    
    shutil.copy(pic_path, obj_path)

In [ ]:
# 移动图片到验证集的相应位置
root =  'D:/1.DataAnalysisPrograme/DeepLearning/Pytroch/Data/CIFAR-10/'

for (id,label) in idLabel_val:
    
    pic_path = root + 'zip/train/' + str(id) + '.png'
    
    obj_path = val_root + '/' + str(label) + '/' + str(id) + '.png'
    
    shutil.copy(pic_path, obj_path)

## 二、数据处理与加载

In [94]:
print('==> Preparing data..')

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]),

    'val': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]),
    'test': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]),
}


==> Preparing data..


In [95]:
data_dir = 'D:/1.DataAnalysisPrograme/DeepLearning/Pytroch/Data/CIFAR/'

image_datasets = {x: torchvision.datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) 
                  for x in ['train', 'val', 'test']}

# 得到可batch的数据格式
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32,
                                             shuffle=True, num_workers=1)
               for x in ['train', 'val']}

testdataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=64,
                                             shuffle=False, num_workers=1)
                   for x in ['test']}


# 得到数据规模
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val','test']}

# 得到分类名
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## 三、建立模型

In [6]:
# 首先定义VGG块，VGG最主要的思想就是多加卷积，且卷积后图的大小不变，图的大小变化靠池化来完成
# input：num_convs：卷积的次数
#       in_channels：输入的通道数
#       out_channels：输出的通道数
#
# output：返回Sequential的VGG模块
def vgg_block(num_convs, in_channels, out_channels):

    net = [nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1), nn.ReLU(True)] # 定义第一层

    for i in range(num_convs-1): # 定义后面的很多层
        net.append(nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1))
        net.append(nn.ReLU(True))
        
    net.append(nn.MaxPool2d(2, 2)) # 定义池化层
    return nn.Sequential(*net)

In [7]:
# 该函数主要用来堆叠VGG块，VGG卷积网络就是由多个VGG块和最后的全连接层实现
# input：num_convs：VGG块中卷积的次数（由数组来表示）
#       channels：通道数（由[输入通道，输出通道]这样的数组所组成的大数组表示
#       out_channels：输出的通道数
#
# output：返回VGG卷积部分的网络
def vgg_stack(num_convs, channels):
    net = []
    for n, c in zip(num_convs, channels):
        in_c = c[0]
        out_c = c[1]
        net.append(vgg_block(n, in_c, out_c))
    return nn.Sequential(*net)

In [85]:
# 建立VGG卷积部分的网络
vgg_net = vgg_stack([1, 1, 2, 2, 2], [[3, 64], [64, 128], [128, 256], [256, 512], [512, 512]])
print(vgg_net)

Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (1): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (2): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (3): Sequential(
    (0): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=

In [96]:
class vggNet(nn.Module):
    
    def __init__(self, num_classes=10):
        super(vggNet, self).__init__()
        
        self.model_name = 'vggNet'
        
        self.features = vgg_net
        
        
        self.fc = nn.Sequential(
            nn.Linear(512, 100),
            nn.ReLU(True),
            nn.Linear(100, 10)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.shape[0], -1)

        x = self.fc(x)
        
        return x

In [97]:
model_ft = vggNet()

## 四、设置必要的参数

In [98]:
# 设置为GPU处理
model_ft = model_ft.to(device)

# 设置损失函数
criterion = nn.CrossEntropyLoss()
# 设置优化算法
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.1)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

## 五、训练阶段

In [99]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    
    # static_dict()用于返回包含模块所有状态的字典，包括参数和缓存。键是参数名称或者缓存名称。
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                since = time.time()
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # 加入一个batch，将得到的数据放置到制定的设备上（GPU或者CPU）
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                # 清空梯度
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                # 以前是使用.data[0]来得到loss，不过官方推荐使用item来转化这里数据
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            
            if phase == 'train':
                time_elapsed = time.time() - since
                print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())



    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [100]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,num_epochs=25)

Epoch 0/24
----------
train Loss: 1.1875 Acc: 0.5825
Training complete in 0m 47s
val Loss: 0.9178 Acc: 0.6740
Epoch 1/24
----------
train Loss: 0.9605 Acc: 0.6644
Training complete in 0m 47s
val Loss: 0.8057 Acc: 0.7340
Epoch 2/24
----------
train Loss: 0.8338 Acc: 0.7126
Training complete in 0m 48s
val Loss: 0.7234 Acc: 0.7455
Epoch 3/24
----------
train Loss: 0.7416 Acc: 0.7459
Training complete in 0m 48s
val Loss: 0.6723 Acc: 0.7740
Epoch 4/24
----------
train Loss: 0.6691 Acc: 0.7731
Training complete in 0m 48s
val Loss: 0.6009 Acc: 0.8065
Epoch 5/24
----------
train Loss: 0.6033 Acc: 0.7934
Training complete in 0m 48s
val Loss: 0.6004 Acc: 0.7950
Epoch 6/24
----------
train Loss: 0.5534 Acc: 0.8121
Training complete in 0m 48s
val Loss: 0.5338 Acc: 0.8205
Epoch 7/24
----------
train Loss: 0.5193 Acc: 0.8241
Training complete in 0m 48s
val Loss: 0.6164 Acc: 0.7885
Epoch 8/24
----------
train Loss: 0.4869 Acc: 0.8348
Training complete in 0m 48s
val Loss: 0.4962 Acc: 0.8385
Epoch 9/24

## 六、预测和提交

In [101]:
# 生成结果存放在preds中
preds = []
for i, (inputs, labels) in enumerate(testdataloaders['test']):
    inputs = inputs.to(device)
    labels = labels.to(device)

    outputs = model_ft(inputs)
    preds.append(torch.max(outputs, 1)[1])

In [102]:
# 将字典转化成numpy格式，以方便上传
labels=[]
for i in range(len(preds)):
    labels.extend(preds[i].cpu().numpy())

In [103]:
# 得到图片的序号
fileindex = []
for i in range(300000):
    fileindex.append(testdataloaders['test'].dataset.imgs[i][0].split('\\')[-1].split('.')[0])

In [104]:
# 将label序号改成label名称
labels = [class_names[x] for x in labels]

In [105]:
# 组合序号和预测结果，存放在.csv中
import pandas as pd

labels = pd.Series(labels)
ids = pd.DataFrame(fileindex)
result = pd.concat([ids, labels], axis=1)
result.columns=['id', 'label']

result['id'] = result['id'].astype('int')

result = result.sort_values(by='id')

result.to_csv('submit.csv', index=None, header=['id', 'label'])

## 七、模型的保存和加载

In [106]:
# 模型保存
torch.save(model_ft, 'cifar10Model.pth')

D:\Complie\Anconda\lib\site-packages\torch\serialization.py:193: UserWarning: Couldn't retrieve source code for container of type vggNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [ ]:
# 模型加载
model_ft = torch.load('cifar10Model.pth') 